<a href="https://colab.research.google.com/github/annyfan/550/blob/master/notebookc80a383385.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Imports
import os
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!wget "https://www.dropbox.com/s/qvo45vf0kc7j41d/ift6758-a20.zip?dl=0" -O data.zip


--2020-12-10 17:31:09--  https://www.dropbox.com/s/qvo45vf0kc7j41d/ift6758-a20.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/qvo45vf0kc7j41d/ift6758-a20.zip [following]
--2020-12-10 17:31:09--  https://www.dropbox.com/s/raw/qvo45vf0kc7j41d/ift6758-a20.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8ec1a584eef8ca3e07e6fc0f34.dl.dropboxusercontent.com/cd/0/inline/BE3R102XIGfDJICygYKsMOEv8w3VkBPuqojeZ_jMihPOHiJ41Ha04gq2qJgN7QRPGmRdfYDVxvzjtRvsmaW4xnd5j8FVVm_mjHoGyDrGoJ79zB0bx_WVXZ4LdNcoMpC34Z0/file# [following]
--2020-12-10 17:31:10--  https://uc8ec1a584eef8ca3e07e6fc0f34.dl.dropboxusercontent.com/cd/0/inline/BE3R102XIGfDJICygYKsMOEv8w3VkBPuqojeZ_jMihPOHiJ41Ha04gq2qJgN7QRPGmRdfYDVxvzjtRvsmaW4xnd5j8FVVm_mj

In [4]:
!unzip data.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_profile_images/profile_images_train/9O6PO4HFOCQ99IHF.png  
  inflating: train_profile_images/profile_images_train/9OGBOKM836P9MHMF.png  
  inflating: train_profile_images/profile_images_train/9OI7OC6A872QJPSH.png  
  inflating: train_profile_images/profile_images_train/9OI8I76J9K28EKQH.png  
  inflating: train_profile_images/profile_images_train/9OPJABRP2617QS4D.png  
  inflating: train_profile_images/profile_images_train/9OT5062HK84PG4N3.png  
  inflating: train_profile_images/profile_images_train/9P0LPSH2KE7K544O.png  
  inflating: train_profile_images/profile_images_train/9P0NH81RQ8CH2ML4.png  
  inflating: train_profile_images/profile_images_train/9P38G80NSFA7KDK6.png  
  inflating: train_profile_images/profile_images_train/9P4FRH3ICREEQL1D.png  
  inflating: train_profile_images/profile_images_train/9PAC0AE37SGQATON.png  
  inflating: train_profile_images/profile_images_train/9PFJD24I7OT7LRS0.png  
  inflating: 

In [5]:
#Visualize the count of number of survivors
sns.set()

In [6]:
# Set random seeds (for reproducibility requirement)
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(67)
np.random.seed(67)
random.seed(67)

In [7]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
test_df.head()



,Id,User Name,Personal URL,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location,Location Public Visibility,User Language,Profile Creation Timestamp,User Time Zone,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image
0,49I3SOKLI2CMNGP4,_FwKTE4dm,NaN,Set,Not verified,fa0a86,fc37c4,0a0101,False,-18000.0,"200 E Broadway, Suite 3D, New York NY 10002",Enabled,en,Mon Jul 20 21:05:24 +0000 2009,Quito,31528,2148,12926,469,business,13.827,3.4062,49I3SOKLI2CMNGP4.png
1,727IRIR59A3P88LK,tDMVTsvWbdpud,NaN,Not set,Not verified,ff0000,e6e6e6,001941,False,36000.0,"ÜT: -28.108029,153.410307",Enabled,en,Wed Aug 05 22:31:34 +0000 2009,Brisbane,42857,262,11678,1806,government,24.019,3.2656,727IRIR59A3P88LK.png
2,LN95SD15SRPCEE8F,k_TUXA297j,https://blob/5ljjaio2rp.com,Set,Verified,0d0101,000000,000000,False,-25200.0,@happyhippiefdn,Enabled,en,Fri Mar 18 18:36:02 +0000 2011,Pacific Time (US & Canada),40237847,392,9395,59247,unknown,26.011,12.1619,LN95SD15SRPCEE8F.png
3,TB11I7F0PN033D4T,HRDr8yt4s7M,NaN,Set,Verified,0000ff,e0ff92,9ae4e8,False,-25200.0,"Austin, TX",??,en,Sun Apr 08 05:45:46 +0000 2007,Pacific Time (US & Canada),230166,702,10507,2695,celebrity,26.127,2.1131,TB11I7F0PN033D4T.png
4,32PSGCK5PATHMR07,PXOI0egSsDqrve,https://blob/t3hjJ0c/Mo.com,Set,Not verified,992f09,ddeef6,ffffff,False,NaN,"New Rochelle, NY",Enabled,en,Mon Jun 04 19:51:04 +0000 2012,NaN,1199,1701,1022,8,unknown,17.878,3.9660,32PSGCK5PATHMR07.png


In [8]:
train_df.isnull().sum()

Id                                                0
User Name                                         0
Personal URL                                   4244
Profile Cover Image Status                       90
Profile Verification Status                       0
Profile Text Color                               66
Profile Page Color                               78
Profile Theme Color                              72
Is Profile View Size Customized?                  0
UTC Offset                                      486
Location                                       1461
Location Public Visibility                        0
User Language                                     0
Profile Creation Timestamp                        0
User Time Zone                                  486
Num of Followers                                  0
Num of People Following                           0
Num of Status Updates                             0
Num of Direct Messages                            0
Profile Cate

In [9]:

# get the number of seconds to the unix epoch start.
df_unix_sec = pd.to_datetime(train_df['Profile Creation Timestamp']).astype(int)/ 10**9

In [10]:
# Finalizing all the pre-processing steps we have seen above.
def pre_process(data):
  #data['Sex'].replace(['male', 'female'], ['1', '0'], inplace=True)   
  #data = data.merge(pd.get_dummies(data['Pclass'], prefix='Pclass'), left_index=True, right_index=True)

  data.loc[data['Personal URL'].notnull(), 'URL']=1
  data.loc[data['Personal URL'].isnull(), 'URL']=0

    
  data['Profile Cover Image Status'] = data['Profile Cover Image Status'].fillna('Not Set').replace(['Set', 'Not set', 'Not Set'], ['1', '0', '0'])

  data['Profile Verification Status'] = data['Profile Verification Status'].fillna('unknown').replace(['Not verified', 'Pending','Verified'], ['0', '1', '2'])
    
  data['Profile Category'] = data['Profile Category'].replace(['business', 'celebrity','government', 'unknown'], ['1', '2', '3', '0'])

  data['Avg Daily Profile Clicks'] = data['Avg Daily Profile Clicks'].fillna(0)
    
  data['Avg Daily Profile Visit Duration in seconds'] = data['Avg Daily Profile Visit Duration in seconds'].fillna(0)

  data['Profile Text Color'] = data['Profile Text Color'].fillna('000000')
  data['Profile Page Color'] = data['Profile Page Color'].fillna('FFFFFF')
  data['Profile Theme Color'] = data['Profile Theme Color'].fillna('FFFFFF')
    
  data['UTC Offset'] = data['UTC Offset'].fillna('0')
  data['Location'] = data['Location'].fillna('unknown')
  data['Location Public Visibility'] = data['Location Public Visibility'].replace(['Disabled', 'Enabled'], ['0', '1'])
  data['User Time Zone'] = data['User Time Zone'].fillna('Central Time (US & Canada)')
  '''

  Color: Convert 16-digit Hex Code to 0-255, Separate Columns RR-GG-BB, then will have numeric features
  Profile Creation Timestamp: convert to long numeric (the year matters)'''

  data['profile_text_r'] = data['Profile Text Color'].str[:2].apply(lambda x: int(x, 16))
  data['profile_text_g'] = data['Profile Text Color'].str[2:4].apply(lambda x: int(x, 16))
  data['profile_text_b'] = data['Profile Text Color'].str[4:].apply(lambda x: int(x, 16))

  data['profile_page_r'] = data['Profile Page Color'].str[:2].apply(lambda x: int(x, 16))
  data['profile_page_g'] = data['Profile Page Color'].str[2:4].apply(lambda x: int(x, 16))
  data['profile_page_b'] = data['Profile Page Color'].str[4:].apply(lambda x: int(x, 16))

  data['profile_theme_r'] = data['Profile Theme Color'].str[:2].apply(lambda x: int(x, 16))
  data['profile_theme_g'] = data['Profile Theme Color'].str[2:4].apply(lambda x: int(x, 16))
  data['profile_theme_b'] = data['Profile Theme Color'].str[4:].apply(lambda x: int(x, 16))

  data['profile_create_sec'] = pd.to_datetime(data['Profile Creation Timestamp']).astype(int)/ 10**9
  
  data['Location Public Visibility'] = data['Location Public Visibility'].replace(['enabled', 'disabled', '??'], ['1', '0', '0'])

  data['Profile Category'] = data['Location Public Visibility'].replace([' '], ['4'])


  return data


train_df_p = pre_process(train_df)
test_df_p = pre_process(test_df) 


In [11]:
train_df_p.head()

,Id,User Name,Personal URL,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location,Location Public Visibility,User Language,Profile Creation Timestamp,User Time Zone,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image,Num of Profile Likes,URL,profile_text_r,profile_text_g,profile_text_b,profile_page_r,profile_page_g,profile_page_b,profile_theme_r,profile_theme_g,profile_theme_b,profile_create_sec
0,AL85S14OMDPF01I9,Mf9vfld4Vfe,NaN,1,2,db1a2c,eaf0f2,e70409,False,39600,unknown,1,en,Thu Nov 27 05:24:59 +0000 2008,Sydney,95763,4289,30809,873,1,14.792,1.5761,AL85S14OMDPF01I9.png,2815,0.0,219,26,44,234,240,242,231,4,9,1.227763e+09
1,HI11QOPD7BLJTO7Q,xl9gaGN0hxM_,NaN,1,2,0099cc,f6ffd1,fff04d,False,0,mumbai,1,en,Fri Jan 15 18:00:46 +0000 2010,Central Time (US & Canada),1018746,289,8150,290,1,8.183,11.2782,HI11QOPD7BLJTO7Q.png,1242,0.0,0,153,204,246,255,209,255,240,77,1.263578e+09
2,JS49LP5P72RI1OQB,d_uiMm,NaN,1,0,1fc2de,efefef,1fc2de,False,-18000,NYC + 70 Countries Worldwide,1,en,Fri Oct 02 20:15:06 +0000 2009,Central Time (US & Canada),13444,1876,4698,227,1,31.823,0.5725,JS49LP5P72RI1OQB.png,1559,0.0,31,194,222,239,239,239,31,194,222,1.254515e+09
3,S0GDSC09MACCLBJP,hfylaRr,https://blob/e/g9pex_vS.com,0,2,050000,616161,00090a,False,-14400,"Indianapolis, In",1,en,Thu Feb 19 14:37:22 +0000 2009,Eastern Time (US & Canada),339168,1148,53216,4035,1,23.052,4.0265,S0GDSC09MACCLBJP.png,6342,1.0,5,0,0,97,97,97,0,9,10,1.235054e+09
4,CRSEMK4QER6LDJSA,hRR1sDGlz5,https://blob/v/Szeo.h4/.com,1,0,58424d,f7f7f7,000000,False,-18000,"777 Beach Blvd. Biloxi, MS",1,en,Tue Mar 31 13:27:52 +0000 2009,Central Time (US & Canada),9215,93,3271,130,1,8.418,3.9229,CRSEMK4QER6LDJSA.png,1078,1.0,88,66,77,247,247,247,0,0,0,1.238506e+09


In [15]:

cols_x = ['URL','Profile Cover Image Status', 'Profile Verification Status','Is Profile View Size Customized?','Location Public Visibility', 'User Language', 'profile_create_sec', 'User Time Zone',
          'Num of Followers', 'Num of People Following', 'Num of Status Updates', 'Num of Direct Messages','Profile Category','Avg Daily Profile Visit Duration in seconds', 
         'Avg Daily Profile Clicks', 'profile_text_r', 'profile_text_g', 'profile_text_b', 'profile_page_r', 'profile_page_g', 'profile_page_b', 'profile_theme_r', 'profile_theme_g', 'profile_theme_b' ]
cols_y = 'Num of Profile Likes'

In [16]:
# Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

# Metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import sys

In [17]:


# frequency encoding
# Color: Frequency Encoding (~1900 colors, they have different encodings)
# User Time Zone: frequency encoding (duplicate feature with UTC-offset)
# User Language: frequency encoding (28 unique)
def trans_freq(x_freq):
  for name in x_freq.columns:
    fe = x_freq.groupby(name).size()/len(x_freq)
    x_freq.loc[:, name] = x_freq[name].map(fe)
 '''
  timezonefe = x_freq.groupby('User Time Zone').size()/len(x_freq)
  x_freq.loc[:, 'User Time Zone'] = x_freq['User Time Zone'].map(timezonefe)

  lfe = x_freq.groupby('User Language').size()/len(x_freq)
  x_freq.loc[:, 'User Language'] =x_freq['User Language'].map(lfe)

  cfe = x_freq.groupby('profile_text_r').size()/len(x_freq)
  x_freq.loc[:, 'profile_text_r'] = x_freq['profile_text_r'].map(cfe)

  cfe = x_freq.groupby('profile_text_g').size()/len(x_freq)
  x_freq.loc[:, 'profile_text_g'] = x_freq['profile_text_g'].map(cfe)

  cfe = x_freq.groupby('profile_text_b').size()/len(x_freq)
  x_freq.loc[:, 'profile_text_b'] = x_freq['profile_text_b'].map(cfe)

  cfe = x_freq.groupby('profile_page_r').size()/len(x_freq)
  x_freq.loc[:, 'profile_page_r'] = x_freq['profile_page_r'].map(cfe)

  cfe = x_freq.groupby('profile_page_g').size()/len(x_freq)
  x_freq.loc[:, 'profile_page_g'] = x_freq['profile_page_g'].map(cfe)

  cfe = x_freq.groupby('profile_text_b').size()/len(x_freq)
  x_freq.loc[:, 'profile_text_b'] = x_freq['profile_text_b'].map(cfe)

  cfe = x_freq.groupby('profile_theme_r').size()/len(x_freq)
  x_freq.loc[:, 'profile_theme_r'] = x_freq['profile_theme_r'].map(cfe)

  cfe = x_freq.groupby('profile_theme_g').size()/len(x_freq)
  x_freq.loc[:, 'profile_theme_g'] = x_freq['profile_theme_g'].map(cfe)

  cfe = x_freq.groupby('profile_theme_b').size()/len(x_freq)
  x_freq.loc[:, 'profile_theme_b'] = x_freq['profile_theme_b'].map(cfe)
  return x_freq
'''

In [33]:

x = train_df_p[cols_x]
y = train_df_p[cols_y]
x['User Time Zone'] = LabelEncoder().fit_transform(x['User Time Zone'])
x['User Language'] = LabelEncoder().fit_transform(x['User Language'])

# split dataset to train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True, random_state=76) #  To guarantee that the split will always be the same 

#one hot encoding + log transformation
x_dum = pd.get_dummies(x)
y_log=np.log(y+1)

X_train_dum,X_test_dum,y_train_log,y_test_log = train_test_split(x_dum, y_log,test_size = 0.1,random_state = 76)

# frequency encoding
x_freq = trans_freq(train_df_p[cols_x])

X_train_freq,X_test_freq,y_train_log,y_test_log = train_test_split(x_freq, y_log,test_size = 0.1,random_state = 76)

y_log=np.log(y+1)
#再整理出一组标准化的数据，通过对比可以看出模型的效果有没有提高
scale_x=StandardScaler()
x1=scale_x.fit_transform(x)
scale_y=StandardScaler()
y=np.array(y).reshape(-1,1)
y1=scale_y.fit_transform(y)
y1=y1.ravel()
X_train1,X_test1,y_train1,y_test1 = train_test_split(x1,y1,test_size = 0.1,random_state = 76)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
np.isnan(X_train_freq.any())
np.all(np.isfinite(X_train_dum))

In [19]:
result = pd.DataFrame(columns = ("Regression","Training","Testing"))
regression, train_scores, test_scores = [],[],[]

In [34]:
r2 = RandomForestRegressor(n_estimators=100,random_state=67)
r2 = r2.fit(X_train,y_train)
regression.append("RandomForestRegressor")

train_scores.append(r2.score(X_train, y_train))
test_scores.append(r2.score(X_test, y_test))
print(mean_squared_log_error(y_test, r2.predict(X_test)))

0.8630845604156514
-0.41470345453739155
5.710896692077522


In [36]:
r2 = r2.fit(X_train_dum,y_train_log)
ypred = r2.predict(X_test_dum)
ypred_original=np.exp(ypred) - 1
ypred_original = np.around(np.maximum(ypred_original, 0))

print(mean_squared_log_error(y_test,ypred_original))

r2 = r2.fit(X_train_freq,y_train_log)
ypred = r2.predict(X_test_freq)
ypred_original=np.exp(ypred) - 1
ypred_original = np.around(np.maximum(ypred_original, 0))

print(mean_squared_log_error(y_test,ypred_original))

r2 = r2.fit(X_train1,y_train1)
ypred = r2.predict(X_test1)
ypred_original=scale_y.inverse_transform(ypred.reshape(-1, 1)).ravel()
ypred_original = np.around(np.maximum(ypred_original, 0))

print(mean_squared_log_error( y_test, ypred_original))


r2 = r2.fit(X_train1,y_train_log)
ypred = r2.predict(X_test1)
ypred_original=np.exp(ypred)- 1
ypred_original = np.around(np.maximum(ypred_original, 0))

print(mean_squared_log_error( y_test, ypred_original))

0.9329113056798882
0.546737492371423
2.9031867417314365
0.9330804374662407
0.5482482233446961
2.8922063284004205
0.8619527624178279
-0.35309173395653426
5.691223046320063
0.863293968611208
-0.4290709069766885
5.714286077952859


In [73]:
  models=[LinearRegression(),
          KNeighborsRegressor(),
          SVR(),
          Ridge(),
          Lasso(),
          MLPRegressor(hidden_layer_sizes=(6, 4, 2,2),  activation='relu', solver='lbfgs', alpha=200, batch_size='auto',learning_rate='adaptive', learning_rate_init=0.0001, power_t=0.8, max_iter=5000, shuffle=True,random_state=67, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,early_stopping=True,beta_1=0.9, beta_2=0.9, epsilon=1e-08),
          DecisionTreeRegressor(),
          ExtraTreeRegressor(),
          RandomForestRegressor(n_estimators=100),
          AdaBoostRegressor(),
          GradientBoostingRegressor(),
          BaggingRegressor(n_estimators=100) ]#XGBRegressor(),
  models_str=['LinearRegression','KNNRegressor','SVR','Ridge','Lasso','MLPRegressor','DecisionTree','ExtraTree','RandomForest','AdaBoost','GradientBoost','Bagging',]
  score_adapt=[]
  for name,model in zip(models_str,models):
    try:
          print('---------------------------------')
          #if name in ['LinearRegression','Ridge','ExtraTree']:   
          print('开始训练模型：'+name+' 平滑处理')
          model=model
          model.fit(X_train_dum,y_train_log)
          y_pred_log=model.predict(X_test_dum)   
          ypred_original=np.exp(y_pred_log) - 1 
          ypred_original = np.around(np.maximum(ypred_original, 0))
          score=model.score(X_test_dum,y_test_log)
          score_adapt.append(str(score)[:5])
          print(name +' 得分:'+str(score))
          print(name +' msle:',mean_squared_log_error(y_test, ypred_original))
    except:
          print(print("Unexpected error:", sys.exc_info()[0]))
    try:
          print('---------------------------------')  
          #elif name in ['SVR','MLPRegressor','Bagging','AdaBoost','KNNRegressor']:
          print('开始训练模型：'+name+' 标准化处理')
              
          model.fit(X_train1,y_train1)
          y_pred1=model.predict(X_test1)
          ypred_original=scale_y.inverse_transform(y_pred1.reshape(-1, 1)).ravel()
          ypred_original = np.around(np.maximum(ypred_original, 0))
          score=model.score(X_test1,y_test1)
          score_adapt.append(str(score)[:5])       
          print(name +' 得分:'+str(score))
          print(name +' msle:',(mean_squared_log_error(y_test, ypred_original)))

    except:
          print(print("Unexpected error:", sys.exc_info()[0]))
    try:  
          print('---------------------------------')      
          print('开始训练模型：'+name+' 频率')
         
          model.fit(X_train_freq,y_train_log)
          y_pred_log=model.predict(X_test_freq)
          y_pred = np.maximum(y_pred, 0)
          ypred_original=np.exp(y_pred_log) - 1 
          ypred_original = np.around(np.maximum(ypred_original, 0))
          score_adapt.append(str(score)[:5])       
          print(name +' 得分:'+str(score))
          print(name +' msle:'+str(mean_squared_log_error(y_test, ypred_original)))

         
    except:
          print(print("Unexpected error:", sys.exc_info()[0]))
    try:    
          print('---------------------------------') 
          print('开始训练模型：'+name+' 普通')
         
          model.fit(X_train,y_train)
          y_pred=model.predict(X_test)
          y_pred = np.around(np.maximum(ypred_original, 0))
          score=model.score(X_test,y_test)
          score_adapt.append(str(score)[:5])       
          print(name +' 得分:'+str(score))
          print(name +' msle:'+str(mean_squared_log_error(y_test, y_pred)))

          print('---------------------------------')
    except:
          print(print("Unexpected error:", sys.exc_info()[0]))

---------------------------------
开始训练模型：LinearRegression 平滑处理
LinearRegression 得分:0.019045159950252244
LinearRegression msle: 4.31360991642848
---------------------------------
开始训练模型：LinearRegression 标准化处理
LinearRegression 得分:0.01663725113193415
LinearRegression msle: 8.718295715694058
---------------------------------
开始训练模型：LinearRegression 频率
LinearRegression 得分:0.01663725113193415
LinearRegression msle:4.2385044629320205
---------------------------------
开始训练模型：LinearRegression 普通
LinearRegression 得分:0.022904909288289144
LinearRegression msle:8.618410158518712
---------------------------------
---------------------------------
开始训练模型：KNNRegressor 平滑处理
KNNRegressor 得分:-0.19721859233380723
KNNRegressor msle: 7.669007030180857
---------------------------------
开始训练模型：KNNRegressor 标准化处理
KNNRegressor 得分:-0.23268247991877924
KNNRegressor msle: 6.3860283353451575
---------------------------------
开始训练模型：KNNRegressor 频率
KNNRegressor 得分:-0.23268247991877924
KNNRegressor msle:7.66900703018

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.60586e-20): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.06892e-19): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.71128e-18): result may not be accurate.
  overwrite_a=True).T


Lasso 得分:0.02278741457772271
Lasso msle:8.612207754354433
---------------------------------
---------------------------------
开始训练模型：MLPRegressor 平滑处理
MLPRegressor 得分:-0.03771238734750337
MLPRegressor msle: 6.647260278810354
---------------------------------
开始训练模型：MLPRegressor 标准化处理
MLPRegressor 得分:-0.0011212020187800764
MLPRegressor msle: 9.931394184576044
---------------------------------
开始训练模型：MLPRegressor 频率
MLPRegressor 得分:-0.0011212020187800764
MLPRegressor msle:6.407389404830305
---------------------------------
开始训练模型：MLPRegressor 普通
MLPRegressor 得分:-0.0014529243301040218
MLPRegressor msle:9.966331397043072
---------------------------------
---------------------------------
开始训练模型：DecisionTree 平滑处理
DecisionTree 得分:0.1267797341992566
DecisionTree msle: 5.593575308806365
---------------------------------
开始训练模型：DecisionTree 标准化处理
DecisionTree 得分:-3.1416180815162678
DecisionTree msle: 5.3242455528632755
---------------------------------
开始训练模型：DecisionTree 频率
DecisionTree 得分:-3.

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

# Set the parameters by cross-validation
tuned_parameters = [{
  'loss': ['ls', 'lad', 'huber', 'quantile'],
  'max_features':['auto', 'sqrt', 'log2']}
]

metric = 'neg_root_mean_squared_error'

cv_strategy = KFold(n_splits=10, shuffle=True)

grid_search = GridSearchCV(
    gdboost, tuned_parameters, scoring=metric, cv=cv_strategy
)
grid_search.fit(X_train_freq, y_train_log);
print('Finished!')

Finished!


In [78]:
print("Best parameters set found on development set:")
print()
print(grid_search.best_params_)

Best parameters set found on development set:

{'loss': 'ls', 'max_features': 'auto'}


In [79]:
y_pred = grid_search.predict(X_test_freq)
ypred_original=np.exp(ypred) - 1
ypred_original = np.maximum(ypred_original, 0)
print(mean_squared_log_error(y_test,ypred_original))

51.35940518550498


In [104]:
gdboost = GradientBoostingRegressor(learning_rate = 0.2, min_samples_split= 5) #loss : {'ls', 'lad', 'huber', 'quantile'}, optional (default='ls')

gdboost = gdboost.fit(X_train_freq,y_train_log)
ypred = gdboost.predict(X_test_freq)
ypred_original=np.exp(ypred) - 1
ypred_original = np.maximum(ypred_original, 0).astype(np.int64)
print(gdboost.score(X_train_freq, y_train_log))
print(gdboost.score(X_test_freq, y_test_log))
print(mean_squared_log_error(y_test,ypred_original))



0.642982169255577
0.5689690816825927
2.758497006378293


In [107]:
gdboost = gdboost.fit(x_freq,y_log)
x_test_freq = trans_freq(test_df_p[cols_x])

predict_test = gdboost.predict(x_test_freq)
ypred_original=np.exp(predict_test) - 1
predict = np.maximum(ypred_original, 0).astype(np.int64)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [108]:
predict

array([3636, 2772,  230, ..., 4010,  707,  293])

In [109]:
# Get indexes
out = pd.DataFrame(test_df['Id'])

# Make predictions
out['Predicted'] = predict # TODO: real predictions

In [110]:
# Check output
out[out['Predicted']  > 0]

,Id,Predicted
0,49I3SOKLI2CMNGP4,3636
1,727IRIR59A3P88LK,2772
2,LN95SD15SRPCEE8F,230
3,TB11I7F0PN033D4T,6538
4,32PSGCK5PATHMR07,173
...,...,...
2495,7SDJE48EFRPPNEJK,212
2496,PRT8RDNG6E86518P,4702
2497,SRNIBIK27BQ2M3PB,4010
2498,6CP232J9R8N84702,707


In [112]:
# Save output
out.to_csv('predictions.csv', index=False)

----------------------------------------------------------------------

In [ ]:

reg_vote = VotingRegressor(
    estimators=[
     
        #('LinearRegression', reg_log), # LinearRegression
        #('RandomForestRegressor', r2), # RandomForestRegressor
        ('MLPRegressor', regr), # MLPRegressor
        ],
    weights=[1])
reg_vote.fit(X_train, y_train)
reg_vote.score(X_test, y_test)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


-0.20095720414158658

In [ ]:
scores = cross_val_score(reg_vote, train_df_p[cols_x], train_df_p[cols_y], cv=10, scoring='neg_mean_squared_error')
print("Voting: loss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Voting: loss: -286851198.76 (+/- 220183969.59)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Feature Engineering Ideas
- Color: Convert 16-digit Hex Code to 0-255, Separate Columns RR-GG-BB, then will have numeric features
- Color: Frequency Encoding (~1900 colors, they have different encodings)
- User Time Zone: frequency encoding (duplicate feature with UTC-offset)
- User Language: frequency encoding (28 unique)
- Profile Creation Timestamp: convert to long numeric (the year matters)

# Comp550 Project Ideas
- https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data
- https://www.kaggle.com/c/quora-insincere-questions-classification/overview